# Dask Array

## Лабораторная работа 11

In [1]:
import dask.array as da
import h5py
import numpy as np

[minutes_n_ingredients_full.hdf5](https://drive.google.com/file/d/1opgIK5XS8aY63G0pXtv6v6rxSHXUquVx/view?usp=sharing)

1. Считайте датасет `recipe` из файла `minutes_n_ingredients_full.hdf5` в виде `dask.array`. Укажите аргумент `chunks=(100_000, 3)` при создании массива. Выведите на экран основную информацию о массиве.

In [85]:
file = h5py.File('minutes_n_ingredients_full.hdf5')
print(file['recipe'].attrs["description"])
recipes = da.from_array(file['recipe'], chunks=(100_000, 3))
recipes

Содержит столбцы id, minutes и n_ingredients из recipes_full.csv


dask.array<array, shape=(2231637, 3), dtype=int64, chunksize=(100000, 3), chunktype=numpy.ndarray>

[h5py doc](https://docs.h5py.org/en/stable/quick.html#core-concepts)

2. Вычислите среднее значение по каждому столбцу, кроме первого. 

In [79]:
for i in range(1, 3):
    print(i, recipes[:, i].mean().compute())

1 30.545404561763405
2 5.419800800936711


3. Исследуйте, как влияет значение аргумента `chunks` при создании `dask.array` на скорость выполнения операции поиска среднего. 

In [78]:
%%time
test_count = da.from_array(file['recipe'], chunks=(100_000, 3))
test_count.mean().compute()

Wall time: 39 ms


375950.1723050538

In [71]:
test_count = da.from_array(file['recipe'], chunks=(1_000_000, 3))

In [72]:
%%time
test_count.mean().compute()

Wall time: 29 ms


375950.1723050538

In [73]:
test_count = da.from_array(file['recipe'], chunks=(100, 3))

In [74]:
%%time
test_count.mean().compute()

Wall time: 9.18 s


375950.1723050538

In [76]:
%%time
test_count = da.from_array(file['recipe'], chunks=(10, 3))
test_count.mean().compute()

Wall time: 1min 43s


375950.1723050538

4. Выберите рецепты, время выполнения которых меньше медианного значения

In [25]:
med = da.median(recipes[:, 1], axis=0).compute()
recipes[recipes[:, 1] < med, :].compute()

array([[1089012,      23,       5],
       [1428572,       0,       5],
       [1400250,      24,       1],
       ...,
       [1029131,      19,       4],
       [1700703,       1,       1],
       [ 713836,       0,       9]], dtype=int64)

5. Посчитайте количество каждого из возможных значений кол-ва ингредиентов

In [26]:
values, counts = da.unique(recipes[:, 2], return_counts=True)
# находит уникальные элементы массива и возвращает их в отсортированном массиве
for value, count in zip(values.compute(), counts.compute()):
    print(value, count)

1 222071
2 224158
3 229388
4 234948
5 240720
6 244360
7 247181
8 246747
9 246816
10 22430
11 19094
12 15165
13 11640
14 8284
15 6014
16 4145
17 2793
18 1913
19 1279
20 852
21 529
22 346
23 244
24 178
25 107
26 68
27 55
28 33
29 22
30 20
31 13
32 5
33 4
34 3
35 4
36 1
37 2
38 1
39 1
40 2
43 1


6. Найдите максимальную продолжительность рецепта. Ограничьте максимальную продолжительность рецептов сверху значением, равному 75% квантилю.

In [27]:
result = da.max(recipes[:, 1]).compute()
result

2147483647

In [61]:
q = recipes.to_dask_dataframe()[1].quantile(0.75).compute()
recipes[recipes[:, 1] > q, 1] = q
result = da.max(recipes[:, 1]).compute()
result

49

7. Создайте массив `dask.array` из 2 чисел, содержащих ваши предпочтения относительно времени выполнения рецепта и кол-ва ингредиентов. Найдите наиболее похожий (в смысле $L_1$) рецепт из имеющихся в датасете.

In [62]:
t = 10
i = 5
my_request = da.from_array([t, i])

In [80]:
diff = my_request - recipes[:, 1:]
diff.compute()
mask = da.abs(diff).sum(axis=1).argmin()
mask.compute()

158

In [70]:
result = recipes[mask].compute()
result

array([2168955,      10,       5], dtype=int64)

8. Работая с исходным файлом в формате `hdf5`, реализуйте алгоритм подсчета среднего значения в блочной форме и вычислите с его помощью среднее значение второго столбца в массиве.

Блочный алгоритм вычислений состоит из двух частей:
1. Загрузка фрагмента за фрагментом данных по `blocksize` элементов и проведение вычислений на этим фрагментом
2. Агрегация результатов вычислений на различных фрагментах для получения результата на уровне всего набора данных

Важно: при работе с `h5py` в память загружаются не все элементы, а только те, которые запрашиваются в данный момент

In [113]:
def block_algorithm(h5_file: str, blocksize: int = 100_000):
    sum_list = []
    cnt_list = []
    for chunk in chunk_read(h5_file, blocksize):
        sum_list.append(chunk.sum()) 
        cnt_list.append(chunk.size)
    return aggregate(sum_list, cnt_list)

def chunk_read(h5_file: str, blocksize: int):
    with h5py.File(h5_file) as f:
        dataset = f['recipe']
        for i in range(0, dataset.shape[0], blocksize):
            yield dataset[i:i + blocksize, 1]
            
def aggregate(sum_list: list, count_list: list):
    return sum(sum_list) / sum(count_list)            

In [114]:
block_algorithm('minutes_n_ingredients_full.hdf5')

1004.2080517575215